In [1]:
import pandas as pd
import numpy as np
import spekpy

In [3]:
def extract_data(data_path: str) -> pd.DataFrame:
    df = pd.read_excel(data_path, skiprows=[i for i in range(43)])
    return df[["KVP (kV)", "Air Kerma (mGy)","Epaisseur min du filtre X-Ray","Type de filtre X-Ray", "Côté carré équivalent (cm)"]].copy()


def get_or_interpolate_absorption_coef(df_coef, energy):
    df_tmp = df_coef.copy()
    abs_coef  = df_tmp.loc[df_tmp["Energy"] == energy]["μen/ρ"]
    if abs_coef.empty:
        df_tmp.loc[len(df_tmp)] = [energy, np.nan, np.nan]
        df_tmp = df_tmp.sort_values("Energy")
        df_tmp = df_tmp.interpolate("linear")
        abs_coef = df_tmp.loc[df_tmp["Energy"] == energy]["μen/ρ"]

    if len(abs_coef) == 0: return np.nan
    return abs_coef.iloc[0]


def set_abs_coef_ratio(beams):
    coef_air = pd.read_excel("./../Data/Air.xlsx") # https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/air.html
    coef_tissu = pd.read_excel("./../Data/SoftTissu.xlsx") # https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/tissue.html
    
    beams["Energy (MeV)"]=[[] for i in range(len(beams["KVP (kV)"]))]

    for i in range(len(beams["KVP (kV)"])):
        s=spekpy.Spek(kvp=beams.loc[i,"KVP (kV)"],th=9)
        if beams.loc[i,"Type de filtre X-Ray"] == "No Filter":
            # filtration inhérente
            s.filter('Al',float(2.5))
            beams.loc[i,"Energy (MeV)"] = s.get_emean() * 0.001
        else:
            my_filters=[# filtration inhérente
                        ('Al',float(2.5)),
                        # filtration additionelle : info RDM
                        ('Al',float(beams.loc[i,"Epaisseur min du filtre X-Ray"].split("\\")[0])),
                        ('Cu',float(beams.loc[i,"Epaisseur min du filtre X-Ray"].split("\\")[1].replace(',','.')))]
            s.multi_filter(my_filters)
            beams.loc[i, "Energy (MeV)"] = s.get_emean() * 0.001

    beams["μen/ρ air"] = [get_or_interpolate_absorption_coef(coef_air, energy) for energy in beams["Energy (MeV)"]]
    beams["μen/ρ tissu"] = [get_or_interpolate_absorption_coef(coef_tissu, energy) for energy in beams["Energy (MeV)"]]
    beams["μen/ρ ratio"] = beams["μen/ρ tissu"] / beams["μen/ρ air"]

    return beams


def compute_skin_dose(beams):
    print(np.mean(beams["Energy (MeV)"]))
    print(np.mean(beams["Côté carré équivalent (cm)"]))
    
    # Coeff pour une energie moyenne de 60 kV, 0.3mm de Cu et un champ 21,6 x 21,7 cm d'après l'article : DOI: 10.1002/acm2.12725
    T_TABLE = 0.73
    BSF = 1.32
    beams["Skin dose (mGy)"] = beams["Air Kerma (mGy)"] * beams["μen/ρ ratio"] * T_TABLE * BSF
    return beams


def main():
    DATA_PATH = "./../Data/Export_20240219163939.xlsx"
    beams = extract_data(DATA_PATH)
    beams = set_abs_coef_ratio(beams)
    beams = compute_skin_dose(beams)


main()
# # hypothèses = tous les faisceaux ont des angles à 0°, la peau est située au niveau du point de référence, la transmission de la table est totale, 

# #print(beams)
# print(sum(beams.dropna()["Skin dose (mGy)"]), "mGy")